In [1]:
from typing import List, Tuple
import torch
import torch.nn.functional as f
import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import clip
from sentence_transformers import SentenceTransformer
from transformers import GPT2LMHeadModel, GPT2Tokenizer


model_name_or_path = "sberbank-ai/rugpt3large_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).cuda()

def ppl(context: str, continuation: str) -> float:
    t = tokenizer(text=[context], text_pair=[continuation], return_token_type_ids=True, return_tensors="pt")
    context_mask = ~t.token_type_ids.bool()

    lossess = []
    while not context_mask.all():
        input_ids = t.input_ids
        target_ids = input_ids.clone()
        target_ids[context_mask] = -100
        with torch.no_grad():
            loss = model(input_ids.to('cuda'), labels=target_ids.to('cuda')).loss
        lossess.append(loss)

        context_mask = context_mask.roll(1)
        context_mask[:, 0] = True
    
    return torch.exp(torch.stack(lossess).sum() / context_mask.size(1)).item()

def cross_ppl(txts: List[str]) -> np.array:
    res = np.zeros((len(txts), len(txts)))
    for i, context in enumerate(txts):
        for j, continuation in enumerate(txts):
            res[i, j] = ppl(context, continuation)
    return torch.tensor(res)

#model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
#model = SentenceTransformer('paraphrase-albert-small-v2')
#model = SentenceTransformer('multi-qa-distilbert-cos-v1')
#model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')
#model = SentenceTransformer('sentence-transformers/gtr-t5-xxl')
#_device = 'cuda' if torch.cuda.is_available() else 'cpu'
#_encoder, _img_transform = clip.load('ViT-B/32', device=_device, jit=False)
#_encoder.eval()

#def calc_adjacency(nodes: List[str]) -> torch.tensor:
#    trankated = [txt[0:70] for txt in nodes]
#    tokens = clip.tokenize(trankated).to(_device)

#    with torch.no_grad():
#        vecs = _encoder.encode_text(tokens).cpu().type(torch.DoubleTensor)
        #vecs = vecs / vecs.norm(dim=1, p=2).unsqueeze(1)
        #vecs = vecs.cpu().numpy().astype(np.float32)
    
    #vecs = torch.tensor(model.encode(nodes))
#    dists = torch.cdist(vecs, vecs)
#    sims = (dists.max(dim=0)[0] - dists)
    #sims = vecs.matmul(vecs.t())
#    return sims / sims.sum(dim=0)

def calc_adjacency(nodes: List[str]) -> torch.tensor:
    #vecs = torch.tensor(model.encode(nodes))
    #dists = torch.cdist(vecs, vecs)
    #sims = (dists.max(dim=0)[0] - dists)
    #sims = vecs.matmul(vecs.t())
    
    dists = cross_ppl(nodes)
    sims = (dists.max(dim=0)[0] - dists)
    return sims / sims.sum(dim=0)

def summorize(text: str) -> List[Tuple[str, int]]:
    nodes = sent_tokenize(text)
    adjacency_matrix = calc_adjacency(nodes)
    qd = torch.linalg.eig(calc_adjacency(nodes))#.eigenvectors[0].real.argsort()
    #print(qd.eigenvalues)
    ranks = qd.eigenvectors[0].real.softmax(dim=0).numpy()
    return ranks, nodes

[nltk_data] Downloading package punkt to /home/vadim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [35]:
text = """
Я хочу рассказать о разработанном мной сервисе реферирования новостных текстов на английском, 
русском и немецком языках. Системы автоматического реферирования (резюмирования) (САР) — тема довольно специфическая и будет интересна в основном тем, 
кто занимается автоматической обработкой языка. 
Хотя идеально исполненный саммарайзер мог бы стать полезным помощником в сферах, где необходимо преодолеть информационный перегруз и 
быстро принять решение о том, какая информация стоит дальнейшего рассмотрения. 
Как обстоит дело? С одной стороны, в процессе поиска аналогов я заметил интересную вещь — большинство найденных мною статей, сервисов, 
репозиториев и пр. датируются самое позднее 2012 годом. На Хабре есть статья на тему автоматического реферирования, опубликованная в 2011 году. 
В этом же году новостное реферирование было последний раз включено в список треков конференции TAC.
С другой стороны, набирают популярность мобильные приложения, которые обрабатывают новостные потоки и представляют пользователю короткие рефераты на выбранные им темы. 
Яркий пример такой востребованности — относительно недавняя (2013 г.) покупка 
Google и Yahoo саммарайзеров-стартапов Wavii и Summly соответственно, а также наличие различных браузер-плагинов, реферирующих веб-страницы (Chrome, Mozilla).
Беглое же тестирование бесплатных он-лайн сервисов реферирования показывает, что большинство из них работает схоже, выдавая одинаково средние результаты, среди которых, пожалуй, в лучшую сторону выделяется Autosummarizer."
""".replace('\n', ' ')

calc_adjacency(nodes)

tensor([[0.1275, 0.1044, 0.1075, 0.1235, 0.1040, 0.1038, 0.1019, 0.1058, 0.0900,
         0.1007],
        [0.1151, 0.1358, 0.1156, 0.1409, 0.1153, 0.1134, 0.1121, 0.1134, 0.1176,
         0.1155],
        [0.1121, 0.1157, 0.1320, 0.1299, 0.1141, 0.1072, 0.1124, 0.1128, 0.1190,
         0.1157],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.1138, 0.1173, 0.1137, 0.1315, 0.1400, 0.1099, 0.1126, 0.1107, 0.1114,
         0.1092],
        [0.0945, 0.0910, 0.0898, 0.0395, 0.0891, 0.1283, 0.0974, 0.0850, 0.0748,
         0.0888],
        [0.0997, 0.0953, 0.0966, 0.0362, 0.0899, 0.1029, 0.1237, 0.0942, 0.0838,
         0.0976],
        [0.1039, 0.1008, 0.1056, 0.0901, 0.1034, 0.1013, 0.1069, 0.1383, 0.1012,
         0.1048],
        [0.1199, 0.1249, 0.1232, 0.1605, 0.1271, 0.1180, 0.1176, 0.1242, 0.1760,
         0.1282],
        [0.1135, 0.1148, 0.1161, 0.1479, 0.1171, 0.1153, 0.1153, 0.1156, 0.1261,
         0.1395]], dtype=tor

In [23]:
ppl

<function __main__.ppl(context: str, continuation: str) -> float>

In [32]:
nodes = sent_tokenize(text, language="russian")

nodes

[' Я хочу рассказать о разработанном мной сервисе реферирования новостных текстов на английском,  русском и немецком языках.',
 'Системы автоматического реферирования (резюмирования) (САР) — тема довольно специфическая и будет интересна в основном тем,  кто занимается автоматической обработкой языка.',
 'Хотя идеально исполненный саммарайзер мог бы стать полезным помощником в сферах, где необходимо преодолеть информационный перегруз и  быстро принять решение о том, какая информация стоит дальнейшего рассмотрения.',
 'Как обстоит дело?',
 'С одной стороны, в процессе поиска аналогов я заметил интересную вещь — большинство найденных мною статей, сервисов,  репозиториев и пр. датируются самое позднее 2012 годом.',
 'На Хабре есть статья на тему автоматического реферирования, опубликованная в 2011 году.',
 'В этом же году новостное реферирование было последний раз включено в список треков конференции TAC.',
 'С другой стороны, набирают популярность мобильные приложения, которые обрабатываю

In [33]:
for txt, sims in zip(nodes, calc_adjacency(nodes)):
    print(txt)
    for i in sims.argsort(descending=True)[1:4]:
        print(f'\t---{round(sims[i].item(), 2)}: {nodes[i]}')
    print('\n\n------------------------\n\n')

 Я хочу рассказать о разработанном мной сервисе реферирования новостных текстов на английском,  русском и немецком языках.
	---0.12: Как обстоит дело?
	---0.11: Хотя идеально исполненный саммарайзер мог бы стать полезным помощником в сферах, где необходимо преодолеть информационный перегруз и  быстро принять решение о том, какая информация стоит дальнейшего рассмотрения.
	---0.11: С другой стороны, набирают популярность мобильные приложения, которые обрабатывают новостные потоки и представляют пользователю короткие рефераты на выбранные им темы.


------------------------


Системы автоматического реферирования (резюмирования) (САР) — тема довольно специфическая и будет интересна в основном тем,  кто занимается автоматической обработкой языка.
	---0.14: Системы автоматического реферирования (резюмирования) (САР) — тема довольно специфическая и будет интересна в основном тем,  кто занимается автоматической обработкой языка.
	---0.12: Яркий пример такой востребованности — относительно не

In [34]:
r, n = summorize(text)

[ (r[i], n[i]) for i in np.flip(r.argsort())]

[(0.13107621351784757,
  'С другой стороны, набирают популярность мобильные приложения, которые обрабатывают новостные потоки и представляют пользователю короткие рефераты на выбранные им темы.'),
 (0.12555980912459597,
  'Яркий пример такой востребованности — относительно недавняя (2013 г.)'),
 (0.12242771916711423,
  'покупка  Google и Yahoo саммарайзеров-стартапов Wavii и Summly соответственно, а также наличие различных браузер-плагинов, реферирующих веб-страницы (Chrome, Mozilla).'),
 (0.10475606597601432,
  ' Я хочу рассказать о разработанном мной сервисе реферирования новостных текстов на английском,  русском и немецком языках.'),
 (0.07634288305813966,
  'Беглое же тестирование бесплатных он-лайн сервисов реферирования показывает, что большинство из них работает схоже, выдавая одинаково средние результаты, среди которых, пожалуй, в лучшую сторону выделяется Autosummarizer."'),
 (0.0752978496336471,
  'Системы автоматического реферирования (резюмирования) (САР) — тема довольно сп

In [36]:
def ppl(context: str, continuation: str) -> float:
    t = tokenizer(text=[context], text_pair=[continuation], return_token_type_ids=True, return_tensors="pt")
    context_mask = ~t.token_type_ids.bool()

    lossess = []
    while not context_mask.all():
        input_ids = t.input_ids
        target_ids = input_ids.clone()
        target_ids[context_mask] = -100
        with torch.no_grad():
            loss = model(input_ids.to('cuda'), labels=target_ids.to('cuda')).loss
        lossess.append(loss)

        context_mask = context_mask.roll(1)
        context_mask[:, 0] = True
    
    return torch.exp(torch.stack(lossess).sum() / context_mask.size(1)).item()

def cross_ppl(txts: List[str]) -> np.array:
    res = np.zeros((len(txts), len(txts)))
    for i, context in enumerate(txts):
        for j, continuation in enumerate(txts):
            res[i, j] = ppl(context, continuation)
    return torch.tensor(res)



In [2]:
context = ['1 2 3', '4'] 
continuation = ['3', '4 5 6 7']

tokenizer.add_special_tokens({'pad_token': '[PAD]'})
t = tokenizer(text=context, text_pair=continuation, padding=True, return_token_type_ids=True, return_tensors="pt")
context_mask = ~t.token_type_ids.bool()

#lossess = []
#while not context_mask.all():
input_ids = t.input_ids
target_ids = input_ids.clone()
target_ids[context_mask] = -100
#    with torch.no_grad():
#        loss = model(input_ids.to('cuda'), labels=target_ids.to('cuda')).loss
#    lossess.append(loss)

#    context_mask = context_mask.roll(1)
#        context_mask[:, 0] = True

input_ids

tensor([[   21,   491,   816,    23, 50257],
        [   24,    24,  1061,  1286,  1409]])

In [74]:
t.attention_mask.bool() & t.token_type_ids.bool()

tensor([[False, False, False,  True, False],
        [False,  True,  True,  True,  True]])

In [78]:
t = tokenizer(text=context, text_pair=continuation, padding=True, return_token_type_ids=True, return_tensors="pt")
t.attention_mask

tensor([[1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1]])